In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd

mp_pose = mp.solutions.pose

dataset_dir = "DATASET/TRAIN"
landmark_list = []
labels = []

pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

for pose_name in os.listdir(dataset_dir):
    pose_folder = os.path.join(dataset_dir, pose_name)
    if not os.path.isdir(pose_folder):
        continue
    for img_name in os.listdir(pose_folder):
        img_path = os.path.join(pose_folder, img_name)
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            landmark_list.append(landmarks)
            labels.append(pose_name)

pose.close()

# Convert to DataFrame
df = pd.DataFrame(landmark_list)
df['label'] = labels
df.to_csv("yoga_landmarks.csv", index=False)
print("Landmarks extracted and saved to yoga_landmarks.csv")


I0000 00:00:1758956910.527727 4602276 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1758956910.589234 4604410 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758956910.597571 4604410 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/baijuyadav/Desktop/hackathon/posedetection/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Premature end of JPEG file
Corrupt JPEG data: premature end of data segment


Landmarks extracted and saved to yoga_landmarks.csv


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import joblib

# Load data
df = pd.read_csv("yoga_landmarks.csv")
X = df.drop("label", axis=1).values
y = df["label"].values

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Save model and label encoder
joblib.dump(clf, "yoga_pose_model.pkl")
joblib.dump(le, "label_encoder.pkl")
print("Model and label encoder saved!")


Accuracy: 0.8947368421052632
              precision    recall  f1-score   support

     downdog       1.00      0.93      0.96        40
     goddess       0.96      0.71      0.81        31
       plank       0.93      0.96      0.95        54
        tree       0.92      0.85      0.88        40
    warrior2       0.75      0.95      0.84        44

    accuracy                           0.89       209
   macro avg       0.91      0.88      0.89       209
weighted avg       0.91      0.89      0.89       209

Model and label encoder saved!
